In [9]:
# pip install numpy==1.24.2

In [10]:
import numpy as np

In [11]:
from keras.models import Sequential

In [12]:
from keras.layers import Dense

In [13]:
from keras.optimizers import Adam

In [14]:
from keras.utils import to_categorical

In [15]:
from keras.layers import Dropout, Flatten

In [16]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [17]:
# pip install opencv-python

In [18]:
import cv2

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
import pickle

In [21]:
import os

In [22]:
import pandas as pd

In [23]:
import random

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [25]:
path = "Dataset" 
labelFile = 'labels.csv' 
batch_size_val=32 
epochs_val=10
imageDimesions = (32,32,3)
testRatio = 0.2    
validationRatio = 0.2 

In [26]:
count = 0
images = []
classNo = []

In [27]:
myList = os.listdir(path)
print("Total Classes Detected:",len(myList))
noOfClasses=len(myList)
print("Importing Classes.....")
for x in range (0,len(myList)):
    myPicList = os.listdir(path+"/"+str(count))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(count)+"/"+y)
        images.append(curImg)
        classNo.append(count)
    print(count, end =" ")
    count +=1
print(" ")
images = np.array(images)
classNo = np.array(classNo)

Total Classes Detected: 43
Importing Classes.....
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42  


In [28]:
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio)

In [29]:
print("Data Shapes")
print("Train",end = "");print(X_train.shape,y_train.shape)
print("Validation",end = "");print(X_validation.shape,y_validation.shape)
print("Test",end = "");print(X_test.shape,y_test.shape)

Data Shapes
Train(22271, 32, 32, 3) (22271,)
Validation(5568, 32, 32, 3) (5568,)
Test(6960, 32, 32, 3) (6960,)


In [30]:
data=pd.read_csv(labelFile)
print("data shape ",data.shape,type(data))

data shape  (43, 2) <class 'pandas.core.frame.DataFrame'>


In [31]:
num_of_samples = []
cols = 5
num_classes = noOfClasses

In [32]:
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img =cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img = grayscale(img)     
    img = equalize(img)      
    img = img/255            
    return img

In [33]:
X_train=np.array(list(map(preprocessing,X_train)))  
X_validation=np.array(list(map(preprocessing,X_validation)))
X_test=np.array(list(map(preprocessing,X_test)))

In [34]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_validation=X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [35]:
dataGen= ImageDataGenerator(width_shift_range=0.1,   
                            height_shift_range=0.1,
                            zoom_range=0.2,  
                            shear_range=0.1,  
                            rotation_range=10) 

In [36]:
dataGen.fit(X_train)
batches= dataGen.flow(X_train,y_train,batch_size=20)
X_batch,y_batch = next(batches)

In [37]:
y_train = to_categorical(y_train,noOfClasses)
y_validation = to_categorical(y_validation,noOfClasses)
y_test = to_categorical(y_test,noOfClasses)

In [40]:
def myModel():
    model= Sequential()
    model.add((Conv2D(60,(5,5),input_shape=(imageDimesions[0],imageDimesions[1],1),activation='relu')))  # ADDING MORE CONVOLUTION LAYERS = LESS FEATURES BUT CAN CAUSE ACCURACY TO INCREASE
    model.add((Conv2D(60, (5,5), activation='relu')))
    model.add(MaxPooling2D(pool_size=(2,2)))
 
    model.add((Conv2D(30, (3,3),activation='relu')))
    model.add((Conv2D(30, (3,3), activation='relu')))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
 
    model.add(Flatten())
    model.add(Dense(500,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses,activation='softmax')) 
    model.compile(Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [42]:
model = myModel()
print(model.summary())
history=model.fit(dataGen.flow(X_train,y_train,batch_size=batch_size_val),steps_per_epoch=len(X_train)//32,epochs=epochs_val,validation_data=(X_validation,y_validation),shuffle=1)
 

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 60)     │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 60)     │        90,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 60)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 10, 10, 30)     │        16,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 30)       │         8,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 500)            │       240,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 43)             │        21,543 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 378,023 (1.44 MB)

 Trainable params: 378,023 (1.44 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


695/695 ━━━━━━━━━━━━━━━━━━━━ 200s 283ms/step - accuracy: 0.1607 - loss: 3.0902 - val_accuracy: 0.7909 - val_loss: 0.7745
Epoch 2/10
  1/695 ━━━━━━━━━━━━━━━━━━━━ 2:47 242ms/step - accuracy: 0.5938 - loss: 1.2103

C:\Users\HP\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


695/695 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.5938 - loss: 1.2103 - val_accuracy: 0.7967 - val_loss: 0.7686
Epoch 3/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 159s 229ms/step - accuracy: 0.5916 - loss: 1.3297 - val_accuracy: 0.8836 - val_loss: 0.3419
Epoch 4/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.8438 - loss: 0.7986 - val_accuracy: 0.8869 - val_loss: 0.3394
Epoch 5/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 174s 203ms/step - accuracy: 0.7216 - loss: 0.8914 - val_accuracy: 0.9147 - val_loss: 0.2810
Epoch 6/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.8125 - loss: 0.4922 - val_accuracy: 0.9247 - val_loss: 0.2590
Epoch 7/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 151s 218ms/step - accuracy: 0.7957 - loss: 0.6593 - val_accuracy: 0.9583 - val_loss: 0.1492
Epoch 8/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.8750 - loss: 0.4568 - val_accuracy: 0.9608 - val_loss: 0.1469
Epoch 9/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 191s 230ms/step - accuracy: 0.8320 - loss: 0.5370 - val

In [46]:
import plotly.express as px
import pandas as pd

# Convert history to a DataFrame for easier plotting
history_df = pd.DataFrame(history.history)

# Plot for Loss
fig_loss = px.line(
    history_df, 
    y=['loss', 'val_loss'], 
    labels={'value': 'Loss', 'epoch': 'Epoch'}, 
    title='Loss per Epoch'
)
fig_loss.update_layout(legend_title_text='Loss Type')
fig_loss.update_traces(mode='lines+markers')
fig_loss.show()

# Plot for Accuracy
fig_accuracy = px.line(
    history_df, 
    y=['accuracy', 'val_accuracy'], 
    labels={'value': 'Accuracy', 'epoch': 'Epoch'}, 
    title='Accuracy per Epoch'
)
fig_accuracy.update_layout(legend_title_text='Accuracy Type')
fig_accuracy.update_traces(mode='lines+markers')
fig_accuracy.show()


In [47]:
model.save("model.h5")